### **Introduction**


**A model is, abstractly:**

* A function that computes something on tensors (a forward pass)
* Some variables that can be updated in response to training

### **Setup**


In [1]:
import tensorflow as tf
from datetime import datetime
import random as ra
import tensorboard as tb

print(tb.__version__)
print(tf.__version__)

2.7.0
2.6.0


### **Defining Models and Layers in TensorFlow**
**Layers are functions with a known `mathematical structure` that can be reused and have `trainable variables`.**

In [2]:
# Creating a simple Model using tf.Module
class Module(tf.Module):
    def __init__(self, name):
        super().__init__(name= name)
        self.tr_var = tf.Variable(5.0, name= 'train_me')
        self.non_tr_var = tf.Variable(5.0, trainable=False , name= 'not_train')

    def __call__(self, x):
        return self.tr_var * x + self.non_tr_var

model = Module('SimpleModel')
model(tf.constant(3.)).numpy()

20.0

In [3]:
# Printing trainable & non_trainable variables
print('Trainable variables: ', model.trainable_variables)
print('Non-Trainable variables', model.non_trainable_variables)
print()
print('All variables: ')
for var in model.variables:
    print(var)

Trainable variables:  (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>,)
Non-Trainable variables (<tf.Variable 'not_train:0' shape=() dtype=float32, numpy=5.0>,)

All variables: 
<tf.Variable 'not_train:0' shape=() dtype=float32, numpy=5.0>
<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>


> **Creating a two-layer linear layer model made out of modules.**

In [4]:
# Creating a dense layer
class Dense(tf.Module):
    def __init__(self, in_fea, out_fea,name=None):
        super().__init__(name)
        self.wei = tf.Variable(
            tf.random.normal([in_fea, out_fea], name= 'weights')
        )
        self.bias = tf.Variable(
            tf.zeros([out_fea], name='bias')
        )

    def __call__(self, x_ten):
        # Computation: Y = Matrix multiplication (x & weights) plus the model bias
        # Computation: tf.nn.relu(Y)
        y = (x_ten @ self.wei) + self.bias
        return tf.nn.relu(y)

In [5]:
class Sequential(tf.Module):
    def __init__(self, name=None):
        super().__init__(name)
        
        self.dense1 = Dense(3, 3, 'InputLayer')
        self.dense2 = Dense(3, 2, 'OutputLayer')
        
    def __call__(self, tx):
        y = self.dense1(tx)
        return self.dense2(y)

my_model = Sequential('AI_Model')
# Model acceptable shape [n x 3]
my_model(tf.random.poisson([2, 3], 5.5)) 
my_model(tf.random.normal([3, 3], mean=17, stddev=3.5)).numpy()


array([[13.139297,  0.      ],
       [18.169588,  0.      ],
       [17.761112,  0.      ]], dtype=float32)

In [6]:
# Printing Model submodules
for mod in my_model.submodules:
    print(mod)
print()

# Printing model Variables
for var in  my_model.variables: 
    print(f'var: {var.numpy()}\n')




var: [0. 0. 0.]

var: [[-0.39903533  2.8584285  -0.28828472]
 [ 0.97410417  1.6884475  -1.436763  ]
 [-1.2081252  -0.09072787 -1.5433416 ]]

var: [0. 0.]

var: [[ 1.1347665   1.6179154 ]
 [ 0.22295702 -1.4868757 ]
 [-0.4721579  -0.1772938 ]]



* ##### **Waiting to create variables**


In [7]:
# The Input shape of the layers is dynamically inferered at the runtime by the size of the incoming tensor..
class dyn_Layer(tf.Module):
    def __init__(self, outCh, name=None):
        super().__init__(name)
        self.is_built = False 
        self.out_channels = outCh
    
    def __call__(self, tx):
        if not self.is_built:
            self.is_built = True
            self.wei = tf.Variable(
                tf.random.normal([tx.shape[-1], self.out_channels]), name='Weights'
            )
            self.bias = tf.Variable(
                tf.zeros([self.out_channels], name='bias')
            )
        
        y = tx @ self.wei + self.bias
        return tf.nn.relu(y)
    
class dyn_SeqModel(tf.Module):
    def __init__(self, name=None):
        super().__init__(name)
        
        self.dense1 = dyn_Layer(12, 'Input_Layer')
        self.dense2 = dyn_Layer(3, 'Output_Layer')
        
    def __call__(self, tx):
        tx = self.dense1(tx)
        return self.dense2(tx)

In [8]:
# Calling the dynamic model on different sets of inputs
for i in range(10):
    in_chs = ra.randint(1,15)
    print('InChanels for this Model', in_chs)
    model = dyn_SeqModel('AdaptiveModel')
    tx = tf.random.uniform([6, 6], minval=16, maxval=32)
    print(model(tx))

# Our model is working...



InChanels for this Model 1
tf.Tensor(
[[  0.       128.71495   69.367096]
 [  0.       156.39355   73.65257 ]
 [  0.       156.72723   79.726494]
 [  0.       167.1547    74.67707 ]
 [  0.       164.22754   80.34685 ]
 [  0.       160.38048   83.29672 ]], shape=(6, 3), dtype=float32)
InChanels for this Model 6
tf.Tensor(
[[ 0.6829262  0.        12.774567 ]
 [ 0.         0.         7.1609077]
 [ 1.662302   0.         3.6560783]
 [ 1.0417137  0.        16.508053 ]
 [ 0.         0.         3.7840614]
 [ 7.532221   0.        16.858067 ]], shape=(6, 3), dtype=float32)
InChanels for this Model 4
tf.Tensor(
[[ 0.         7.5912714 59.240707 ]
 [ 0.        28.03339   19.110558 ]
 [ 0.        25.52978   35.754654 ]
 [ 0.         3.8108416 85.12776  ]
 [ 0.         0.        67.24708  ]
 [ 0.        29.605312  26.638424 ]], shape=(6, 3), dtype=float32)
InChanels for this Model 5
tf.Tensor(
[[149.4383     0.         0.      ]
 [159.87935    0.         0.      ]
 [ 80.19273    0.         0.      ]

### **Saving Weights**

In [10]:
# Create a dir to save weights
chk_path = 'Weights_checkpoint'
checkpoint = tf.train.Checkpoint(model=model)

# Using this Command, the weights are stored into the file.
checkpoint.write(chk_path)
tf.train.list_variables(chk_path)

# Creating a new Model ans assigning the SAME WEIGHTS
new_mod = dyn_SeqModel('Dynamic_Seq2')
checkpoint2 = tf.train.Checkpoint(model=new_mod)

# Reloading the previous weights



[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('model/dense1/bias/.ATTRIBUTES/VARIABLE_VALUE', [12]),
 ('model/dense1/wei/.ATTRIBUTES/VARIABLE_VALUE', [6, 12]),
 ('model/dense2/bias/.ATTRIBUTES/VARIABLE_VALUE', [3]),
 ('model/dense2/wei/.ATTRIBUTES/VARIABLE_VALUE', [12, 3])]

### **Saving Functions**

* ##### **Creating a saved Model**


### **Keras Models and Layers**

* ##### **Keras Layers**

* ##### **The build step**


* ##### **Keras Models**


### **Saving Keras Model**

### **`Sonnet`: Alternate deep-learning API for building ML Models**
> **By DeepMind,**
> **Based on tf.Module**